Write a python program which creates a database using SQLITE3 module with following instructions..

->Create a table called MyRestaurants with the following attributes: 
	Name of the restaurant: a varchar field
	Type of food they make: a varchar field
	Distance (in minutes) from your house: an integer
	Date of your last visit: a varchar field, interpreted as date
	Whether you like it or not: an integer, interpreted as a Boolean

->Insert at least five tuples using the SQL INSERT command five (or more) times. 
	You should insert at least one restaurant you liked, at least one restaurant you did not like, and at least one restaurant where you leave the iLike field NULL.

->Write a SQL query to return all restaurants in your table.

->Modify your SQL query such that it prints "I liked it" or "I hated it" for each restaurant you liked or not. # HINT- USE SQL CASE COMMAND 
 	Note that you are not allowed to modify the table on disk. You should be able to answer this question using only a SELECT statement.
 	A solution that creates and uses an extra table, howerver, will be accepted.

->Write a SQL query that returns all restaurants that you like, but have not visited since more than 3 months ago. 

In [3]:
import mysql.connector
from mysql.connector import MySQLConnection
from configparser import ConfigParser

def read_db_config(filename='data/config.ini', section='mysql'):
    """ Read database configuration file and return a dictionary object
    :param filename: name of the configuration file
    :param section: section of database configuration
    :return: a dictionary of database parameters
    """
    # create parser and read ini configuration file
    parser = ConfigParser()
    parser.read(filename)
 
    # get section, default to mysql
    db = {}
    if parser.has_section(section):
        items = parser.items(section)
        for item in items:
            db[item[0]] = item[1]
    else:
        raise Exception('{0} not found in the {1} file'.format(section, filename))
 
    return db # dictionary containing key-value pairs of the credentials to your Mysql server connection 

from mysql.connector import MySQLConnection, Error
 
def connect():
    """ Connect to MySQL database """
 
    db_config = read_db_config()
    conn = None
    try:
        print('Connecting to MySQL database...')
        conn = MySQLConnection(**db_config) # kwargs 
 
        if conn.is_connected():
            print('Connection established.')
        else:
            print('Connection failed.')
 
    except Error as error:
        print(error)
 
    finally:
        if conn is not None and conn.is_connected():
            conn.close()
            print('Connection closed.')


Connecting to MySQL database...
Connection established.
Connection closed.


In [11]:
'''>Create a table called MyRestaurants with the following attributes: 
	Name of the restaurant: a varchar field
	Type of food they make: a varchar field
	Distance (in minutes) from your house: an integer
	Date of your last visit: a varchar field, interpreted as date
	Whether you like it or not: an integer, interpreted as a Boolean'''
def create_table():
    db_con=read_db_config()
    connection=MySQLConnection(**db_con)
    cursor=connection.cursor()
    query="CREATE TABLE RES(name varchar(225), type varchar(225), distance INTEGER, date varchar(225), liked INTEGER)"
    cursor.execute(query)
    connection.close()


In [15]:
'''->Insert at least five tuples using the SQL INSERT command five (or more) times. 
	You should insert at least one restaurant you liked, at least one restaurant you did not like, and at least one restaurant where you leave the iLike field NULL.'''

def insert():
    db_con=read_db_config()
    connection=MySQLConnection(**db_con)
    cursor=connection.cursor()
    query = "INSERT INTO RES(name, type, distance, date, liked) VALUES(%s, %s, %s, %s, %s)"
    restaurents =[("Dominos", "Pizza", 10, "April 22, 2022", 1),
                    ("McDonalds", "Fast Food", 5, "January 22, 2022", 1),
                    ("Pizza Hut", "Pizza", 15, "September 04, 2008", 0),
                    ("Subway", "Sandwich", 7, "October 13, 2021", 1),
                    ("Taco Bell", "Mexican", 11, "November 27, 2022", None),
                    ("Wendy's", "Fast Food", 9, "December 31, 2021", 1),
                    ("CTR", "Dosa", 12, "January 14, 2022", 1)]
    cursor.executemany(query, restaurents)
    connection.commit()
    connection.close()

In [18]:
'''->Write a SQL query to return all restaurants in your table.'''

def query():
    db_con=read_db_config()
    connection=MySQLConnection(**db_con)
    cursor=connection.cursor()
    query="SELECT * FROM RES"
    cursor.execute(query)
    for row in cursor:
        print(row)

In [33]:
'''->Modify your SQL query such that it prints "I liked it" or "I hated it" for each restaurant you liked or not. # HINT- USE SQL CASE COMMAND 
 	Note that you are not allowed to modify the table on disk. You should be able to answer this question using only a SELECT statement.
 	A solution that creates and uses an extra table, howerver, will be accepted.'''

def do_they_like():
    db_con=read_db_config()
    connection=MySQLConnection(**db_con)
    cursor=connection.cursor()
    query='''SELECT name, CASE
            WHEN liked=0 THEN 'I Hated it'
            WHEN liked=1 THEN 'I Liked it'
            ELSE 'I left it blank'
            END as result FROM RES'''
    # query = '''SELECT name,CASE WHEN liked=0 THEN 'I hated it' WHEN liked=1 THEN 'I liked it' ELSE 'No like No hate' END as result FROM RES'''
    cursor.execute(query)
    for row in cursor:
        print(row)
    connection.close()

In [35]:
'''Write a SQL query that returns all restaurants that you like, but have not visited since more than 3 months ago. '''
import datetime

def have_i_gone():
    db_con=read_db_config()
    connection=MySQLConnection(**db_con)
    cursor=connection.cursor()
    query='''SELECT name FROM RES WHERE liked=1 AND date < %s'''
    now=datetime.datetime.now()
    three_months_ago=now-datetime.timedelta(days=90)
    cursor.execute(query, (three_months_ago,))
    for row in cursor:
        print(row)
    connection.close()

In [37]:
 
 
if __name__ == '__main__':
    # connect()
    # create_table()
    # insert()
    # query()
    # do_they_like()
    have_i_gone()
